In [1]:
!pip install transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.2 MB/s eta 0:00:00


In [2]:
import gradio as gr
from transformers import pipeline
import random

In [3]:
caption_generator=pipeline("text-generation",model="gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
emoji_dict = {
    "POSITIVE": ["😊", "😄", "😁", "🥳", "🤩", "❤️", "👍"],
    "NEGATIVE": ["😢", "😠", "😞", "💔", "😭", "👎"],
    "NEUTRAL": ["😐", "🤔", "🙃", "😶", "🫥"]
}

sentiment_pipeline=pipeline("sentiment-analysis")

def get_emojis(text):
  label=sentiment_pipeline(text)[0]['label'].lower()
  return ''.join(random.sample(emoji_dict.get[label,["😐"]],3))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
def get_hashtags(prompt, platform):
  words=prompt.lower().split()
  tags=["#" + word.replace(" ","") for word in words if len(word) > 3]

  platform_tags={
      "Instagram":["#instadaily","#igers","#picoftheday"],
      "LinkedIN":["#career","#leadership","#networking"],
      "Twitter":["#tweet","#trending","#news"]
  }

  return " ".join(tags[:5] + random.sample(platform_tags[platform], 2))


In [6]:
def generate_post(prompt, platform):
  caption=caption_generator(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']

  emojis= get_emojis(caption)

  hashtags= get_hashtags(prompt, platform)

  return caption.strip(), emojis, hashtags

In [7]:
interface=gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="enter keyword or theme"),
        gr.Radio(["Instagram","LinkedIn","Twitter"], label="Choose Platform")
    ],
    outputs=[
        gr.Textbox(label="Caption"),
        gr.Textbox(label="Emojis"),
        gr.Textbox(label="Hashtags")
    ],
    title="Social Media Post & Caption Generator",
    description="Generate catchy captions, relevant hashtags, and emojis absed on your theme!"
)
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://547128599c6c58fb5e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


![Gradio Interface](Demo.jpg)